In [34]:
import requests

try:
    from bs4 import BeautifulSoup
except:
    !pip install bs4
    from bs4 import BeautifulSoup
    
import re

from cromulent.model import factory
    
from lib import linkedart as la


baseURI = "https://collections.ashmolean.org/collection/"


ash_file = '/Users/tanya/Documents/Code/linkedart/linked-art/data/ashmolean/ash.html'

html_text = open(ash_file, 'r').read()
soup = BeautifulSoup(html_text, 'html.parser')




for artwork in soup.find_all('div',attrs={"class":"list-inner"}):
    
    title = artwork.find('h3').string
    id = artwork.find('h3').find('a').get('href')
    id = id[-6:]
    creator = [{"id": "", "name":"John Ruskin","role":"Artist"}]
    accession_number = artwork.find('span').string
    accession_number = accession_number.replace("Accession no. ", "")
    image_url = artwork.find("img").get("src")
    date_created = artwork.find("p").string
    date_created = date_created.replace("Date: ","")
    
    objProp = {
        "classification":"",
        "homepage":"https://collections.ashmolean.org/collection/browse-9148/object/" + id,
        "id":id,
        "title":title,
        "creator":creator,
        "accession_number":accession_number,
        "image_url":image_url,
        "date_created":date_created,
        "date_created_earliest": "",
        "date_created_latest": "",
        "current_owner" : {"name":"Ashmolean Museum",
                                "location":"Oxford, England, UK",
                                "type": "http://vocab.getty.edu/aat/300312281" ,
                                "type_label": ""}
    }
    
    # look for year in value 
    years = re.findall('(\d{4})', date_created)
    if len(years) == 1:
                        
        objProp["date_created_earliest"] = years[0]
        objProp["date_created_latest"] = years[0]
    elif len(years) == 2:                
        objProp["date_created_earliest"] = years[0]
        objProp["date_created_latest"] = years[1]
    else:
        # use Ruskins birth/death years
        objProp["date_created_earliest"] = 1819
        objProp["date_created_latest"] = 1900
    
    
    object_uri = baseURI + id
    
    objLA = la.createObjDesc(objProp,la.objTypes,object_uri)
    
    # write to file 
    text_file = open("./data/ashmolean/json/" + objProp["id"] + ".json", "wt")
    n = text_file.write(factory.toString(objLA, compact=False))
    text_file.close()
    